In [68]:
import datetime
import datetime
from pathlib import Path
from types import SimpleNamespace
import pickle
import os
import dotenv

from azure.storage.blob import BlobServiceClient
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
import seaborn
import sklearn
import sklearn.metrics
import sklearn.cross_decomposition
import sklearn.linear_model
import statsmodels
import statsmodels.api
import warnings

In [69]:
import sys
sys.path.append("/Users/missd/Desktop/6. Semester/6. Model Deployment Maintenance/MDM - Python Projekt/src/")

In [70]:
import mdm_python.data.db_entsoe as db_entsoe
import mdm_python.data.create_model as energy_model

In [71]:
model_directory = Path("../data/models").resolve()
plot_directory = Path("../src/mdm_python/backend_server/static/pictures").resolve()

In [72]:
def connect_to_azure():
    # Load environment variables from .env file
    dotenv.load_dotenv()
    
    # Get Azure-Connection-STring
    azure_connection_string = os.getenv("AZURE_STORAGE_CONNECTION_STRING")
    return azure_connection_string

In [65]:
def load_models():
    azure_connection_string = connect_to_azure()
    blob_service_client = BlobServiceClient.from_connection_string(azure_connection_string)

    containers = blob_service_client.list_containers(include_metadata=True)
    suffix = 0
    for container in containers:
        existingContainerName = container['name']
        if existingContainerName.startswith("energy-model"):
            parts = existingContainerName.split("-")
            newSuffix = int(parts[-1])
            if newSuffix > suffix:
                suffix = newSuffix

    container_name = f"energy-model-{suffix}"
    print(f'loading from container: {container_name}')


    container_client = blob_service_client.get_container_client(container_name)
    blob_list = container_client.list_blobs()

    models = dict()
    for blob in blob_list:
        print("\t" + blob.name)
        content = container_client.download_blob(blob.name).readall()
        model = pickle.loads(content)
        models[blob.name]=model["fitted_model"]

    print(models)
    return models

In [66]:
models = load_models()

NameError: name 'dotenv' is not defined

In [60]:
models['wind.pickle']

(<statsmodels.tsa.arima.model.ARIMAResultsWrapper at 0x28aa5ba10>,)

In [5]:
energy_data = db_entsoe.extract_daily_energy()

In [20]:
def energy_forecast(raw_data, energy_type, model, forecast_horizon):

    data_set = energy_model.prepare_raw_data(raw_data)
    data_set = energy_model.detrend_and_deseasonalize_data(data_set)
    
    p, d, q = model["model"].order
    
    modelling = energy_model.ARIMA_model(data_set[energy_type].detrended_values, p, 0, q)
    forecast = modelling.get_forecast(steps=forecast_horizon).summary_frame()

    baseline = data_set[energy_type].baseline.iloc[-1]

    offset = data_set[energy_type].offset
    untransform = lambda v: 10**(v-offset) if offset is not None else v
    
    forecast_df = pd.DataFrame(index=forecast.index, data={
        energy_type:untransform(forecast["mean"] + baseline),
        f"lower_{energy_type}":untransform(forecast["mean_ci_lower"] + baseline),
        f"upper_{energy_type}":untransform(forecast["mean_ci_upper"] + baseline),
    })
    
    return forecast_df

In [21]:
def plot_forecast(raw_data, energy_types, models, forecast_horizon):
    plot_paths = {}

    for energy_type in energy_types:
        
        model = models[energy_type]
        
        forecast_df = energy_forecast(raw_data=raw_data, energy_type=energy_type, model=model, forecast_horizon=forecast_horizon)
        
        # Plot the observed data
        plt.figure(figsize=(10, 6))
        plt.plot(raw_data.index, raw_data[energy_type], label=f'Observed {energy_type}', color='blue')
        
        # Plot the forecasted values
        plt.plot(forecast_df.index, forecast_df[energy_type], label=f'Forecasted {energy_type}', color='red')
        
        # Add shades for uncertainty intervals
        plt.fill_between(forecast_df.index, forecast_df[f'lower_{energy_type}'], forecast_df[f'upper_{energy_type}'], color='lightcoral', alpha=0.3)
        
        # Add labels, legend, and title
        plt.xlabel('Date')
        plt.ylabel('Value')
        plt.title(f'Observed and Forecasted Data with Confidence Intervals for {energy_type}')
        plt.legend()
        plt.grid(True)

        # Define the path for saving the plot
        plot_path = plot_directory/f"{energy_type}_forecast.png"

        # Save the plot
        plt.savefig(plot_path)
        plt.close()  # Important to free memory

        # Store the relative or absolute path in the dictionary
        plot_paths[energy_type] = plot_path

    return plot_paths

In [13]:
energy_types = ["wind", "solar", "nuclear"]

In [19]:
plot_forecast(energy_data, energy_types, 300)

{'wind': PosixPath('/Users/missd/Desktop/6. Semester/6. Model Deployment Maintenance/MDM - Python Projekt/src/mdm_python/backend_server/static/pictures/wind_forecast.png'),
 'solar': PosixPath('/Users/missd/Desktop/6. Semester/6. Model Deployment Maintenance/MDM - Python Projekt/src/mdm_python/backend_server/static/pictures/solar_forecast.png'),
 'nuclear': PosixPath('/Users/missd/Desktop/6. Semester/6. Model Deployment Maintenance/MDM - Python Projekt/src/mdm_python/backend_server/static/pictures/nuclear_forecast.png')}